Based on Tetris implementation in Python, (c) 2010 "Kevin Chabowski" <kevin@kch42.de>

https://gist.github.com/silvasur/565419/d9de6a84e7da000797ac681976442073045c74a4

In [ ]:
import torch
from torch.optim import Adam
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
COLUMNS = 8
ROWS = 16

In [ ]:
TETRIS_SHAPES = [
    [[1, 1, 1],
     [0, 1, 0]],

    [[0, 2, 2],
     [2, 2, 0]],

    [[3, 3, 0],
     [0, 3, 3]],

    [[4, 0, 0],
     [4, 4, 4]],

    [[0, 0, 5],
     [5, 5, 5]],

    [[6, 6, 6, 6]],

    [[7, 7],
     [7, 7]]
]

COLORS = {
    1: '#fff200', # yellow
    2: '#ff0000', # red
    3: '#0000ff', # blue
    4: '#aa00ff', # violet
    5: '#15bb40', # green
    6: '#ff9900', # orange
    7: '#00bbb0', # turquoise
}

In [ ]:
def rotate_clockwise(shape):
    return [
        [shape[y][x] for y in range(len(shape))] for x in range(len(shape[0]) - 1, -1, -1)]


def check_collision(board, shape, offset):
    off_x, off_y = offset
    for cy, row in enumerate(shape):
        for cx, cell in enumerate(row):
            try:
                if cell and board[cy + off_y][cx + off_x]:
                    return True
            except IndexError:
                return True
    return False


def remove_row(board, row):
    del board[row]
    return [[0 for i in range(COLUMNS)]] + board


def join_matrixes(mat1, mat2, mat2_off):
    off_x, off_y = mat2_off
    for cy, row in enumerate(mat2):
        for cx, val in enumerate(row):
            mat1[cy + off_y - 1][cx + off_x] += val
    return mat1


def new_board():
    board = [[0 for x in range(COLUMNS)]
             for y in range(ROWS)]
    board += [[1 for x in range(COLUMNS)]]
    return board

In [ ]:
class TetrisState():
    def __init__(self):
        self.board = new_board()
        self.gameover = False
        self.new_stone()
     
    
    def new_stone(self):
        self.stone = random.choice(TETRIS_SHAPES)
        self.stone_x = int(COLUMNS / 2 - len(self.stone[0]) / 2)
        self.stone_y = 0

        if check_collision(self.board, self.stone, (self.stone_x, self.stone_y)):
            self.gameover = False
  
    def move(self, delta_x):
        if not self.gameover:
            new_x = self.stone_x + delta_x
            if new_x < 0:
                new_x = 0
            if new_x > config['cols'] - len(self.stone[0]):
                new_x = config['cols'] - len(self.stone[0])
            if not check_collision(self.board, self.stone, (new_x, self.stone_y)):
                self.stone_x = new_x
  
  
    def drop(self):
        if not self.gameover:
            self.stone_y += 1
            if check_collision(self.board, self.stone, (self.stone_x, self.stone_y)):
                self.board = join_matrixes(self.board, self.stone, (self.stone_x, self.stone_y))
                self.new_stone()
                while True:
                    for i, row in enumerate(self.board[:-1]):
                        if 0 not in row:
                            self.board = remove_row(
                                self.board, i)
                            break
                    else:
                        break

    def rotate_stone(self):
        if not self.gameover:
            new_stone = rotate_clockwise(self.stone)
            if not check_collision(self.board,
                                   new_stone,
                                   (self.stone_x, self.stone_y)):
                self.stone = new_stone
  
    def move_left(self):
        self.move(1)

    def move_right(self):
        self.move(-1)

    def turn_cw(self):
        new_stone = rotate_clockwise(self.stone)
        if check_collision(self.board, new_stone, (self.stone_x, self.stone_y)):
            return False
        else:
            self.stone = new_stone
            return True

    def turn_ccw(self):
        new_stone = rotate_clockwise(rotate_clockwise(rotate_clockwise(self.stone)))
        if check_collision(self.board, new_stone, (self.stone_x, self.stone_y)):
            return False
        else:
            self.stone = new_stone
            return True
    
    def rewards(self):
        # reward ideas
        # default reward: 0
        # * making points, 1,2,3,4 lines at once adds increasing amounts of reward
        # * if doing nothing will give a point, use reward for points but decreease by how many steps are still required to make a point
        # * putting a block somewhere that does not mask anoher block gives a reward. Probably not needed?

        # what happens when current stone falls done without change
        # unaccessible gaps = bad
        # removing rows = good
        return {
            'left': 0.0,
            'right': 0.0,
            'turn_cw': 0.0,
            'turn_ccw': 0.0,
            'nothing': 0.0,
        }

    def draw(self):
        fig = plt.figure()
        ax = fig.add_subplot(111, aspect='equal')
        plt.xlim(0, COLUMNS)
        plt.ylim(0, ROWS)
        for row in range(ROWS):
            for column in range(COLUMNS):
                color = self.board[row][column]
                if color != 0:
                     ax.add_patch(patches.Rectangle((column, ROWS - row - 1),
                                  1.0, 1.0, facecolor=COLORS[color], edgecolor='#000000'))
        plt.show()

state = TetrisState()
for i in range(50):
    state.drop()
state.draw()

In [ ]:
NUM_CLASSES = 4 # turn left, turn right, move left, move right
LEARING_RATE = 0.001

input_size = COLUMNS * ROWS * 2
HIDDEN_SIZE = 50

class Model2Linear(nn.Module):
    def __init__(self):
        super().__init__()
        self.h1 = nn.Linear(input_size, HIDDEN_SIZE)
        self.h2 = nn.Linear(HIDDEN_SIZE, HIDDEN_SIZE)
        self.h3 = nn.Linear(HIDDEN_SIZE, NUM_CLASSES)

    def forward(self, x):
        x = x.data.view(-1, input_size)
        x = self.h1(x)
        x = F.relu(x)
        x = self.h2(x)
        x = F.relu(x)
        x = self.h3(x)
        x = F.softmax(x, dim=1)
        return x

In [ ]:
def evalulate(model):
    model.eval()
    loss = 0.0
    for data, labels in validation_loader:
        predictions_per_class = model(data.cuda())
        _, highest_prediction_class = predictions_per_class.max(1)
        loss += F.nll_loss(predictions_per_class, labels.cuda())
    return loss/len(validation_loader)

def learn():
    model = Model1Linear().cuda()
    optimizer = Adam(params=model.parameters(), lr=LEARING_RATE)

    for epoch in range(1000):
        model.train()
        for data, labels in loader:
            predictions_per_class = model(data.cuda())
            highest_prediction, highest_prediction_class = predictions_per_class.max(1)

            # how good are we? compare output with the target classes
            loss = F.nll_loss(predictions_per_class, labels.cuda())

            model.zero_grad() # ???
            loss.backward() # backpropagate
            optimizer.step()
        
        validation_loss = evalulate(model)
        print(f'Epoch: {epoch}, Loss: {validation_loss.item()}')
        
    return model
%time model = learn()